#### Simple Gen AI APP Using Langchain

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [3]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

In [28]:
loader=WebBaseLoader("https://en.wikipedia.org/wiki/Narendra_Modi")
loader

In [29]:
docs=loader.load()
docs

[Document(page_content='\n\n\n\nNarendra Modi - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nEarly life and education\n\n\n\n\n\n\n\n\n2\nEarly political career\n\n\n\n\n\n\n\n\n3\nChief Minister of Gujarat (2001–2014)\n\n\n\n\nToggle Chief Minister of Gujarat (2001–2014) subsection\n\n\n\n\

In [30]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [31]:
documents

[Document(page_content='Narendra Modi - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nEarly life and education\n\n\n\n\n\n\n\n\n2\nEarly political career\n\n\n\n\n\n\n\n\n3\nChief Minister of Gujarat (2001–2014)\n\n\n\n\nToggle Chief Minister of Gujarat (2001–2014) subsection\n\n\n\n\n\n3.1\n

In [32]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [33]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [34]:
vectorstoredb

In [36]:
## Query From a vector db
query="2002 Gujarat violence"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'2002 Gujarat violence\nMain article: 2002 Gujarat violence'

In [37]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [60]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context, if you don't find any relevant information, say "I don't know".
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the following question based only on the provided context, if you don\'t find any relevant information, say "I don\'t know".\n<context>\n{context}\n</context>\n\n\n'))])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x13f8139a0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x12fb92ec0>, model_name='gpt-4o', openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

In [61]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"Was there any democratic backsliding in India under Narendra Modi?",
    "context":[Document(page_content="Under Modi's tenure, India has experienced democratic backsliding and has shifted towards an authoritarian style of government, with a cult of personality centred around him.[f] As prime minister, he has received consistently high approval ratings within India. Modi has been described as engineering a political realignment towards right-wing politics. He remains a highly controversial figure domestically and internationally over his Hindu nationalist beliefs and handling of the Gujarat violence, which have been cited as evidence of a majoritarian and exclusionary social agenda.[g]", metadata={'source': 'https://en.wikipedia.org/wiki/Narendra_Modi', 'title': 'Narendra Modi - Wikipedia', 'language': 'en'}),
]
})

"I don't know."

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [62]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [63]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [64]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10dace170>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the following question based only on the provided context, if you don\'t find any relevant information, say "I don\'t know".\n<context>\n{context}\n</context>\n\n\n'))])
            | ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x13f8139a0>, async_client=<openai

In [65]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"Tell me about LangChain technology"})
response['answer']

"I don't know."

In [66]:

response

{'input': 'Tell me about LangChain technology',
 'context': [Document(page_content='Further reading', metadata={'source': 'https://en.wikipedia.org/wiki/Narendra_Modi', 'title': 'Narendra Modi - Wikipedia', 'language': 'en'}),
  Document(page_content='This page was last edited on 2 January 2026, at 18:44\xa0(UTC).\nText is available under the Creative Commons Attribution-ShareAlike 4.0 License;\nadditional terms may apply. By using this site, you agree to the Terms of Use and Privacy Policy. Wikipedia® is a registered trademark of the Wikimedia Foundation, Inc., a non-profit organization.\n\n\nPrivacy policy\nAbout Wikipedia\nDisclaimers\nContact Wikipedia\nLegal & safety contacts\nCode of Conduct\nDevelopers\nStatistics\nCookie statement\nMobile view\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\nToggle the table of contents\n\n\n\n\n\n\n\nNarendra Modi\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n146 languages\n\n\nAdd topic', met

In [67]:
response['context']

[Document(page_content='Further reading', metadata={'source': 'https://en.wikipedia.org/wiki/Narendra_Modi', 'title': 'Narendra Modi - Wikipedia', 'language': 'en'}),
 Document(page_content='This page was last edited on 2 January 2026, at 18:44\xa0(UTC).\nText is available under the Creative Commons Attribution-ShareAlike 4.0 License;\nadditional terms may apply. By using this site, you agree to the Terms of Use and Privacy Policy. Wikipedia® is a registered trademark of the Wikimedia Foundation, Inc., a non-profit organization.\n\n\nPrivacy policy\nAbout Wikipedia\nDisclaimers\nContact Wikipedia\nLegal & safety contacts\nCode of Conduct\nDevelopers\nStatistics\nCookie statement\nMobile view\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\nToggle the table of contents\n\n\n\n\n\n\n\nNarendra Modi\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n146 languages\n\n\nAdd topic', metadata={'source': 'https://en.wikipedia.org/wiki/Narendra_Modi